In [ ]:
from openai import OpenAI
import json
from llama_index.core.schema import TextNode
import json
import time
from pathlib import Path
from typing import List, Dict
import os
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.core.schema import TextNode
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core.llms import ChatMessage
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
import pandas as pd 
import json
# os.environ["OPENAI_API_KEY"] = "sk-R55EltjXK0FAaHbqhPV1OHtWifZXEBJRbDXQS4JZUk4uxwvo"
# os.environ["OPENAI_API_BASE"] = "https://api.shubiaobiao.com/v1"
# openai.api_key = "sk-R55EltjXK0FAaHbqhPV1OHtWifZXEBJRbDXQS4JZUk4uxwvo"
# openai.base_url = 'https://api.shubiaobiao.com/v1'

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), verbose=True)  # 读取本地 .env 文件，里面定义了 OPENAI_API_KEY

In [ ]:
URI = "./milvus_filter_demo.db"  

In [ ]:
with open('/Users/vitol/vscode/firm_perf/firm_perf_code/转JSON/json文件汇总/merged.json', 'r', encoding='utf-8') as file:

    json_list = json.load(file)

In [ ]:
def create_nodes(raw_data: List[Dict]) -> List[TextNode]:
    """添加ID稳定性保障"""
    nodes = []
    for entry in raw_data:
        j2str = []
        content_str = json.dumps(entry['text'],ensure_ascii=False)

        id_e = entry["id"]



        try:
            node = TextNode(
                    text=content_str,
                    id_=id_e,  # 显式设置稳定ID
                    metadata={
                        "公司全称": entry['text']['公司全称'],
                        "公司简称": entry['text']['公司简称'],
                        "股票代码": entry['text']['股票代码'],
                        "行业分类": entry['text']['申万行业分类'],
                    }
                )
            nodes.append(node)
        except Exception as e:
            print(entry['id'][:9])
            node = TextNode(
                    text=content_str,
                    id_=id_e,  # 显式设置稳定ID
                    metadata={
                        "股票代码": entry['id'][:9],
                    }
                )
            nodes.append(node)
            continue
    
    print(f"生成 {len(nodes)} 个文本节点（ID示例：{nodes[0].id_}）")
    return nodes

In [ ]:
nodes = create_nodes(json_list)

In [ ]:
# client = OpenAI(
#     api_key="sk-R55EltjXK0FAaHbqhPV1OHtWifZXEBJRbDXQS4JZUk4uxwvo",  # 替换为您的API密钥
#     base_url="https://api.shubiaobiao.com/v1"  # 替换为您的代理URL
# )
# def get_embeddings(texts, model="text-embedding-ada-002", dimensions=None):
#     '''封装 OpenAI 的 Embedding 模型接口'''
#     if model == "text-embedding-ada-002":
#         dimensions = None
#     if dimensions:
#         data = client.embeddings.create(
#             input=texts, model=model, dimensions=dimensions).data
#     else:
#         data = client.embeddings.create(input=texts, model=model).data
#     return [x.embedding for x in data]
# test_query = ["测试文本"]
# vec = get_embeddings(test_query)[0]
# print(f"Total dimension: {len(vec)}")
# print(f"First 10 elements: {vec[:10]}")

In [ ]:
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from pymilvus import model

# openai_ef = model.dense.OpenAIEmbeddingFunction(
#     model_name='text-embedding-ada-002',
#     api_key='',
#     base_url='https://api.shubiaobiao.com/v1'
# )
# embed_model = OpenAIEmbedding(
#     model="text-embedding-ada-002",
#     base_url="https://api.shubiaobiao.com/v1",
#     api_key=""
# )
embed_model = HuggingFaceEmbedding(
        model_name=r"/Users/vitol/vscode/firm_perf/firm_perf_code/embed"
    )
vector_store = MilvusVectorStore(
    uri=URI,
    # token=TOKEN,
    collection_name="companyinfo",  # Change collection name here
    dim=512,  # Vector dimension depends on the embedding model
    overwrite=True,  # Drop collection if exists

)
storage_context = StorageContext.from_defaults(vector_store=vector_store)



In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context,embed_model=embed_model)

In [ ]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

In [ ]:
retriever = index.as_retriever(similarity_top_k=1)
result_nodes = retriever.retrieve("平安银行")

for node in result_nodes:
    print(node.text)
    # print(node.metadata)
    print("\n")

In [ ]:
from openai import OpenAI
import json
from llama_index.core.schema import TextNode
import json
import time
from pathlib import Path
from typing import List, Dict
import os
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.core.schema import TextNode
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core.llms import ChatMessage
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
import pandas as pd 
import json

In [ ]:
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from pymilvus import model
embed_model = HuggingFaceEmbedding(
        model_name=r"/Users/vitol/vscode/firm_perf/firm_perf_code/embed"
    )

In [ ]:
URI = "/Users/vitol/vscode/firm_perf/firm_perf_code/转JSON/合并json code/milvus_filter_demo.db"  

In [ ]:
from pymilvus import MilvusClient

In [ ]:
client = MilvusClient(uri=URI)

In [ ]:
# 重新连接到现有的向量存储
vector_store = MilvusVectorStore(
    uri=URI, 
    dim=512,
    collection_name="companyinfo",
    overwrite=False  # 设置为 False 以使用现有集合
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store,embed_model=embed_model)

In [ ]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

In [ ]:
retriever = index.as_retriever(similarity_top_k=1)
result_nodes = retriever.retrieve("平安银行")

for node in result_nodes:
    print(node.text)
    # print(node.metadata)
    print("\n")

In [ ]:
from llama_index.llms.openai import OpenAI

# 创建 LlamaIndex 的 OpenAI LLM 实例
llm = OpenAI(
    model="gpt-4o",
    api_base='https://api.shubiaobiao.com/v1',
    api_key=''  
)
response = llm.complete("Hello, how are you?")
print(response)

In [ ]:


from llama_index.llms.openai import OpenAI

# 创建 LlamaIndex 的 OpenAI LLM 实例
llm = OpenAI(
    model="gpt-4o",
    api_base='https://api.ai-gaochao.cn/v1',
    api_key=''  
)



# 在查询引擎中使用这个 LLM
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=1
    )
res = query_engine.query("平安银行注册地址？")
print(res)

iface = gr.Interface(fn=chat_with_robot, inputs="text", outputs="text", title="我的吐槽机器人")

iface.launch(share=True)

In [ ]:

import streamlit as st
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.vector_stores.milvus.utils import BM25BuiltInFunction
from pymilvus import MilvusClient

# 初始化组件
@st.cache_resource
def init_query_engine():
    # 创建 LlamaIndex 的 OpenAI LLM 实例
    llm = OpenAI(
        model="gpt-4o",
        api_base='https://api.ai-gaochao.cn/v1',
        api_key=''  
    )
    
    # client = MilvusClient(uri="milvus_filter_demo.db")
    # 连接到本地 Milvus Lite（sqlite 文件）
    # 注意：uri 需要用 sqlite:/// 前缀，且路径要绝对路径
    vector_store = MilvusVectorStore(
        uri="./milvus_filter_demo.db",
        collection_name="companyinfo",
        dim=512,
        overwrite=True,
    )
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents([], storage_context=storage_context)
    
    # 创建查询引擎
    query_engine = index.as_query_engine(
        llm=llm,
        similarity_top_k=1
    )
    
    return query_engine

# Streamlit 界面
st.title("智能问答系统")
st.write("基于 Milvus 和 LlamaIndex 的 RAG 系统")

# 初始化查询引擎
try:
    query_engine = init_query_engine()
except Exception as e:
    st.error("Milvus 连接失败，请检查本地数据库文件路径和权限。错误信息：{}".format(str(e)))
    st.stop()

# 用户输入
user_question = st.text_input("请输入您的问题：", placeholder="例如：平安银行注册地址？")

if st.button("提交") and user_question:
    with st.spinner("正在查询..."):
        try:
            # 执行查询
            response = query_engine.query(user_question)
            
            # 显示结果
            st.success("查询完成！")
            st.write("**回答：**")
            st.write(response)
            
        except Exception as e:
            # 针对 Milvus 连接失败的报错做特殊提示
            if "Fail connecting to server on localhost:19530" in str(e):
                st.error("Milvus 连接失败，请确认您使用的是本地 Milvus Lite（sqlite 文件），而不是需要启动服务的 Milvus Server。\n"
                         "如需使用本地 sqlite 文件，请确保 uri 以 sqlite:/// 开头，且文件路径正确。\n"
                         "原始错误信息：{}".format(str(e)))
            else:
                st.error(f"查询出错：{str(e)}")



In [ ]:
import tempfile
import os

# 使用系统临时目录，通常有读写权限
temp_dir = tempfile.gettempdir()
db_path = os.path.join(temp_dir, "milvus_filter_demo.db")
db_path
